In [27]:
import os
import warnings
  
# Dependency imports
import matplotlib
#matplotlib.use("Agg")

import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from keras.datasets import mnist
import keras
from tensorflow.keras.layers import Conv2D, Activation, MaxPool2D, Flatten, Dense, Dropout


In [11]:
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.python.keras import backend

from tensorflow.keras import layers
from tensorflow_probability import layers as tfp_layers

In [4]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [28]:
# model = keras.Sequential(
#     [
#         keras.Input(shape=input_shape),
#         layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
#         layers.MaxPooling2D(pool_size=(2, 2)),
#         layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
#         layers.MaxPooling2D(pool_size=(2, 2)),
#         layers.Flatten(),
#         layers.Dropout(0.5),
#         layers.Dense(num_classes, activation="softmax"),
#     ]
# )

model = keras.Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu',input_shape=input_shape))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))



model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [7]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 16s 37ms/step - loss: 0.7805 - accuracy: 0.7581 - val_loss: 0.0834 - val_accuracy: 0.9770
Epoch 2/15
422/422 [==============================] - 17s 39ms/step - loss: 0.1207 - accuracy: 0.9625 - val_loss: 0.0566 - val_accuracy: 0.9847
Epoch 3/15
422/422 [==============================] - 17s 41ms/step - loss: 0.0840 - accuracy: 0.9736 - val_loss: 0.0450 - val_accuracy: 0.9870
Epoch 4/15
422/422 [==============================] - 17s 40ms/step - loss: 0.0719 - accuracy: 0.9773 - val_loss: 0.0405 - val_accuracy: 0.9887
Epoch 5/15
422/422 [==============================] - 16s 38ms/step - loss: 0.0594 - accuracy: 0.9815 - val_loss: 0.0374 - val_accuracy: 0.9905
Epoch 6/15
422/422 [==============================] - 18s 42ms/step - loss: 0.0589 - accuracy: 0.9827 - val_loss: 0.0378 - val_accuracy: 0.9902
Epoch 7/15
422/422 [==============================] - 16s 38ms/step - loss: 0.0526 - accuracy: 0.9843 - val_loss: 0.0332 - val_accuracy:

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.024861343204975128
Test accuracy: 0.9922000169754028


# Leveraging Dropout for UQ 

In [24]:
model.Input
    

AttributeError: 'Sequential' object has no attribute 'Input'

In [30]:

f = K.function([model.layers[0].input, backend.symbolic_learning_phase()],
               [model.layers[-1].output])
# result = []
# n_iter=10
# for i in range(n_iter):
#     result.append(f([x.reshape(1,x_test.shape[0],x_test.shape[1],x_test.shape[2]),True]))



ValueError: Input tensors to a Functional must come from `tf.keras.Input`. Received: 0 (missing previous layer metadata).

In [24]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        tfp_layers.Convolution2DFlipout(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        tfp_layers.Convolution2DFlipout(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

logits = model(images)
labels_distribution = tfd.Categorical(logits=logits)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_flipout_6 (Conv2DFlip (None, 26, 26, 32)        608       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_flipout_7 (Conv2DFlip (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

/Users/nicholas.rossi/miniconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
